In [ ]:
import os
import random
from typing import List, Dict, Any, Tuple
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.retrievers import BM25Retriever
from langchain.retrievers.multi_query import MultiQueryRetriever
import gradio as gr

class HealthInsuranceAssistant:
    def __init__(self, knowledge_base_path: str, groq_api_key: str,
                 chunk_size: int = 1000, chunk_overlap: int = 200):
        """
        Initialize the Health Insurance Assistant with enhanced RAG capabilities

        Args:
            knowledge_base_path (str): Path to the text file knowledge base
            groq_api_key (str): API key for Groq
            chunk_size (int): Size of text chunks for processing
            chunk_overlap (int): Overlap between chunks
        """
        # Setup Environment and API Key
        os.environ["GROQ_API_KEY"] = groq_api_key

        # Store chunking parameters
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

        # Initialize LLM (assuming ChatGroq is a specific model from Groq)
        self.llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0.1,
            max_tokens=200
        )

        # Initialize embeddings
        self.embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

        # Load and process knowledge base
        self.knowledge_base = self._load_and_process_knowledge_base(knowledge_base_path)

        # Initialize components
        self.setup_rag_pipeline()
        self.setup_guardrails()
        self.setup_conversational_elements()

        # Initialize conversation history
        self.conversation_history = []

    def _load_and_process_knowledge_base(self, path: str) -> List[Document]:
        """
        Load and process the knowledge base from text file into documents with chunking

        Args:
            path (str): Path to the knowledge base text file

        Returns:
            List[Document]: Processed documents
        """
        # Load text file
        with open(path, 'r', encoding='utf-8') as file:
            raw_text = file.read()

        # Text splitter for chunking
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            length_function=len,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
        )

        # Split text into sections based on headers (assuming sections are marked with #)
        sections = []
        current_category = "General"
        current_topic = "General Information"
        current_content = []

        for line in raw_text.split('\n'):
            if line.startswith('# '):
                # Save previous section if it exists
                if current_content:
                    sections.append({
                        'category': current_category,
                        'topic': current_topic,
                        'content': '\n'.join(current_content)
                    })
                    current_content = []
                current_category = line[2:].strip()
            elif line.startswith('## '):
                # Save previous section if it exists
                if current_content:
                    sections.append({
                        'category': current_category,
                        'topic': current_topic,
                        'content': '\n'.join(current_content)
                    })
                    current_content = []
                current_topic = line[3:].strip()
            elif line.strip():
                current_content.append(line)

        # Add the last section
        if current_content:
            sections.append({
                'category': current_category,
                'topic': current_topic,
                'content': '\n'.join(current_content)
            })

        # Convert to documents
        documents = []
        for section in sections:
            metadata = {
                "category": section["category"],
                "topic": section["topic"],
                "source": "insurance_knowledge_base"
            }

            # Combine content with metadata
            content = (f"Category: {section['category']}\n"
                      f"Topic: {section['topic']}\n"
                      f"Content: {section['content']}")

            # Create chunks
            chunks = text_splitter.create_documents(
                [content],
                metadatas=[metadata]
            )
            documents.extend(chunks)

        return documents

    def setup_rag_pipeline(self):
        """
        Setup enhanced RAG pipeline with multiple retrievers and reranking
        """
        # Setup vector store
        self.vectorstore = FAISS.from_documents(
            self.knowledge_base,
            self.embeddings
        )

        # Setup BM25 retriever
        self.bm25_retriever = BM25Retriever.from_documents(self.knowledge_base)

        # Setup multi-query retriever for query expansion
        self.multi_query_retriever = MultiQueryRetriever.from_llm(
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 3}),
            llm=self.llm
        )

        # Setup main QA chain
        self.setup_retrieval_qa_chain()

    def setup_guardrails(self):
        """
        Configure safety and ethical guardrails specific to insurance
        """
        self.sensitive_topics = [
            'fraud', 'legal advice', 'specific policy recommendations',
            'personal medical information', 'financial details', 'claims disputes',
            'immediate coverage needs', 'emergency situations', 'policy cancellations'
        ]

        self.emergency_response = """
        For immediate coverage needs or emergency situations, please:
        1. Contact your insurance provider's emergency hotline
        2. Call emergency services if medical attention is needed
        3. Document all interactions and keep receipts
        4. Contact your insurance agent/broker during business hours
        """

    def setup_conversational_elements(self):
        """
        Setup insurance-specific greeting patterns and responses
        """
        self.greeting_patterns = [
            'hi', 'hello', 'hey', 'good morning', 'good afternoon',
            'good evening', 'help', 'insurance help', 'coverage question'
        ]

        self.greeting_responses = [
            "Hello! I'm your health insurance assistant. How can I help you understand your coverage options?",
            "Hi there! I'm here to help you navigate health insurance topics. What would you like to know?",
            "Welcome! I can help you understand health insurance terms, coverage, and processes. What questions do you have?",
            "Greetings! I'm your insurance information guide. How can I assist you today?"
        ]

    def setup_retrieval_qa_chain(self):
        """
        Setup Retrieval QA Chain with advanced configurations
        """
        qa_template = """
        You are a knowledgeable health insurance information assistant focused on helping people understand their insurance options, coverage, claims, and related processes. Your role is to provide clear, accurate information while maintaining appropriate boundaries.

        Context: {context}

        Question: {question}

        Key Guidelines:
        1. Provide clear explanations of insurance terms, policies, and processes
        2. Use simple language while explaining insurance terminology when necessary
        3. Never provide specific policy recommendations or legal advice
        4. Maintain a professional and helpful tone
        5. Acknowledge the complexity of insurance policies when appropriate
        6. Focus on general insurance education and information
        7. Encourage consultation with insurance providers or agents for specific policy questions
        8. Explain insurance concepts thoroughly but concisely
        9. Address common insurance misconceptions
        10. Provide context for insurance terms and processes
        11. Direct users to appropriate resources for urgent coverage issues
        12. Respect privacy and sensitive financial information
        13. Validate concerns while maintaining professional boundaries
        14. Include references to official insurance resources when appropriate
        15. Clarify limitations of general insurance information

        Response:
        """

        PROMPT = PromptTemplate(
            template=qa_template,
            input_variables=["context", "question"]
        )

        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self._hybrid_retriever,
            chain_type_kwargs={"prompt": PROMPT},
            return_source_documents=True
        )

    def _hybrid_retriever(self, query: str) -> List[Document]:
        """
        Hybrid retrieval combining vector search, BM25, and query expansion

        Args:
            query (str): User query

        Returns:
            List[Document]: Retrieved documents
        """
        # Get results from different retrievers
        vector_docs = self.vectorstore.similarity_search(query, k=3)
        bm25_docs = self.bm25_retriever.get_relevant_documents(query)
        expanded_docs = self.multi_query_retriever.get_relevant_documents(query)

        # Combine and deduplicate results
        all_docs = vector_docs + bm25_docs + expanded_docs
        seen_contents = set()
        unique_docs = []

        for doc in all_docs:
            if doc.page_content not in seen_contents:
                seen_contents.add(doc.page_content)
                unique_docs.append(doc)

        return unique_docs

    def process_query(self, query: str) -> str:
        """
        Process a query from the user

        Args:
            query (str): User's question

        Returns:
            str: Response to the query
        """
        # Check for greetings
        if any(greeting in query.lower() for greeting in self.greeting_patterns):
            response = random.choice(self.greeting_responses)
            return response

        # Retrieve and process answer
        answer = self.qa_chain.run(query)
        return answer

    def save_conversation_history(self):
        """Saves conversation history."""
        with open("conversation_history.txt", "a") as file:
            for entry in self.conversation_history:
                file.write(f"{entry['query']}: {entry['response']}\n")

    def reset_conversation_history(self):
        """Resets conversation history."""
        self.conversation_history = []

# Gradio Interface for interaction
def insurance_assistant_interface(query: str):
    assistant = HealthInsuranceAssistant(knowledge_base_path="/content/query,response,category,complexity.txt",
                                         groq_api_key="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")
    response = assistant.process_query(query)
    return response

# Set up Gradio interface
def main():
    gr.Interface(fn=insurance_assistant_interface,
                 inputs="text",
                 outputs="text",
                 title="Health Insurance Assistant",
                 description="An AI assistant that helps users understand health insurance terms, coverage, and processes.").launch()

if __name__ == "__main__":
    main()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e663a8234468907c4b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install --upgrade torch gradio langchain sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1
